In [1]:
import sys
sys.path.append('projects/periodical-clustering')

from utils import *

In [2]:
import pandas as pd
import os
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), '2010s', 'clustering_results', 'cluster_df.parquet'))
cluster_df

data_dir:  /home/zqlyu2/projects/periodical-clustering/data


,Scopus_label,movMF_label,movMF_distance,x_val,y_val,kmeans_label,kmeans_distance,skm_label,skm_distance,spectral_label,n2v_kmeans_label,cm_kmeans_label,gnn_kmeans_label,bert_kmeans_label
202381698,Multidisciplinary,22,0.445886,-67.928200,15.572327,17,0.628846,20,0.444711,24,3,21,0,0
137773608,Multidisciplinary,22,0.590942,-68.405334,-55.633186,17,0.735654,20,0.559494,24,20,21,0,3
125754415,Multidisciplinary,22,0.574571,-68.448853,-55.613579,17,0.705024,20,0.550081,24,20,21,0,0
3880285,Multidisciplinary,23,0.618842,-68.407288,-55.634430,8,0.724859,17,0.610582,24,20,21,0,3
111155417,Chemistry,23,0.220853,-54.506985,-61.217068,11,0.495787,12,0.198758,14,4,24,21,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2764485818,Medicine,21,0.268070,-28.969574,34.819569,18,0.542531,14,0.251015,8,18,1,19,1
83454320,Arts and Humanities,25,0.034777,78.609909,31.736822,13,0.251599,3,0.056642,21,5,1,14,1
16507453,Arts and Humanities,6,0.113656,89.206772,17.625090,13,0.307089,3,0.128660,21,5,1,14,1
121509672,Agricultural and Biological Sciences,5,0.179111,-36.757057,-0.591017,21,0.430040,16,0.207435,10,5,1,10,1


# Sankey

In [95]:
# 读取数据
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s", 'clustering_results', "cluster_df.parquet"))
method1, method2 = "Scopus", "kmeans"

# 计算每个method1_label类别的样本数的10%
method1_counts = cluster_df.groupby(f"{method1}_label").size()
min_thresholds = {label: int(count * 0.1) for label, count in method1_counts.items()}

# 统计流量信息，按 Scopus_label 和 skm_label 分组计数
flow_data = (
    cluster_df.groupby([f"{method1}_label", f"{method2}_label"])
    .size()
    .reset_index(name="flow")
    .apply(lambda row: [row[f"{method1}_label"], str(row[f"{method2}_label"]), row["flow"]], axis=1)
    .tolist()
)

# 过滤流量数据，只保留大于对应method1_label阈值的条目
filtered_flow_data = [entry for entry in flow_data if entry[2] > min(min_thresholds[entry[0]], 50)]

for entry in filtered_flow_data:
    if method1 == "Scopus":
        entry[0] = str(entry[0])
    else:
        entry[0] = method1 + "_" + str(entry[0])
    entry[1] = method2 + "_" + entry[1]
filtered_flow_data

data_dir:  /home/zqlyu2/projects/periodical-clustering/data


[['Agricultural and Biological Sciences', 'kmeans_8', 616],
 ['Agricultural and Biological Sciences', 'kmeans_16', 104],
 ['Agricultural and Biological Sciences', 'kmeans_21', 526],
 ['Arts and Humanities', 'kmeans_0', 54],
 ['Arts and Humanities', 'kmeans_13', 1107],
 ['Arts and Humanities', 'kmeans_19', 56],
 ['Arts and Humanities', 'kmeans_23', 202],
 ['Biochemistry, Genetics and Molecular Biology', 'kmeans_17', 538],
 ['Business, Management and Accounting', 'kmeans_24', 591],
 ['Chemical Engineering', 'kmeans_11', 90],
 ['Chemical Engineering', 'kmeans_14', 22],
 ['Chemistry', 'kmeans_11', 332],
 ['Computer Science', 'kmeans_2', 291],
 ['Computer Science', 'kmeans_5', 228],
 ['Decision Sciences', 'kmeans_2', 12],
 ['Decision Sciences', 'kmeans_15', 19],
 ['Decision Sciences', 'kmeans_24', 47],
 ['Dentistry', 'kmeans_3', 136],
 ['Earth and Planetary Sciences', 'kmeans_12', 447],
 ['Economics, Econometrics and Finance', 'kmeans_0', 56],
 ['Economics, Econometrics and Finance', 'kmean

In [96]:
node_appearance_order = []
for entry in filtered_flow_data:
    if entry[0] not in node_appearance_order:
        node_appearance_order.append(entry[0])
    if entry[1] not in node_appearance_order:
        node_appearance_order.append(entry[1])
node_appearance_order

['Agricultural and Biological Sciences',
 'kmeans_8',
 'kmeans_16',
 'kmeans_21',
 'Arts and Humanities',
 'kmeans_0',
 'kmeans_13',
 'kmeans_19',
 'kmeans_23',
 'Biochemistry, Genetics and Molecular Biology',
 'kmeans_17',
 'Business, Management and Accounting',
 'kmeans_24',
 'Chemical Engineering',
 'kmeans_11',
 'kmeans_14',
 'Chemistry',
 'Computer Science',
 'kmeans_2',
 'kmeans_5',
 'Decision Sciences',
 'kmeans_15',
 'Dentistry',
 'kmeans_3',
 'Earth and Planetary Sciences',
 'kmeans_12',
 'Economics, Econometrics and Finance',
 'kmeans_1',
 'Energy',
 'Engineering',
 'kmeans_6',
 'Environmental Science',
 'Health Professions',
 'kmeans_9',
 'kmeans_20',
 'kmeans_22',
 'Immunology and Microbiology',
 'kmeans_18',
 'Materials Science',
 'Mathematics',
 'Medicine',
 'kmeans_4',
 'kmeans_7',
 'Multidisciplinary',
 'Neuroscience',
 'Nursing',
 'Pharmacology, Toxicology and Pharmaceutics',
 'Physics and Astronomy',
 'Psychology',
 'Social Sciences',
 'kmeans_25',
 'Veterinary']

In [97]:
# 生成一个字典，key为kmeans_label，value为该kmeans_label中出现次数最多的Scopus_label
label_counts = cluster_df.groupby([f'{method1}_label', 'Scopus_label']).size().unstack(fill_value=0)
method12Scopus = label_counts.idxmax(axis=1).to_dict()
if method1 != "Scopus":
    method12Scopus = {f"{method1}_{k}": v for k, v in method12Scopus.items()}
method12Scopus

{'Agricultural and Biological Sciences': 'Agricultural and Biological Sciences',
 'Arts and Humanities': 'Arts and Humanities',
 'Biochemistry, Genetics and Molecular Biology': 'Biochemistry, Genetics and Molecular Biology',
 'Business, Management and Accounting': 'Business, Management and Accounting',
 'Chemical Engineering': 'Chemical Engineering',
 'Chemistry': 'Chemistry',
 'Computer Science': 'Computer Science',
 'Decision Sciences': 'Decision Sciences',
 'Dentistry': 'Dentistry',
 'Earth and Planetary Sciences': 'Earth and Planetary Sciences',
 'Economics, Econometrics and Finance': 'Economics, Econometrics and Finance',
 'Energy': 'Energy',
 'Engineering': 'Engineering',
 'Environmental Science': 'Environmental Science',
 'Health Professions': 'Health Professions',
 'Immunology and Microbiology': 'Immunology and Microbiology',
 'Materials Science': 'Materials Science',
 'Mathematics': 'Mathematics',
 'Medicine': 'Medicine',
 'Multidisciplinary': 'Multidisciplinary',
 'Neuroscien

In [98]:
# 生成一个字典，key为kmeans_label，value为该kmeans_label中出现次数最多的Scopus_label
label_counts = cluster_df.groupby([f'{method2}_label', 'Scopus_label']).size().unstack(fill_value=0)
method22Scopus = label_counts.idxmax(axis=1).to_dict()
method22Scopus = {f"{method2}_{k}": v for k, v in method22Scopus.items()}
method22Scopus

{'kmeans_0': 'Social Sciences',
 'kmeans_1': 'Social Sciences',
 'kmeans_2': 'Computer Science',
 'kmeans_3': 'Medicine',
 'kmeans_4': 'Medicine',
 'kmeans_5': 'Engineering',
 'kmeans_6': 'Physics and Astronomy',
 'kmeans_7': 'Medicine',
 'kmeans_8': 'Agricultural and Biological Sciences',
 'kmeans_9': 'Psychology',
 'kmeans_10': 'Engineering',
 'kmeans_11': 'Chemistry',
 'kmeans_12': 'Earth and Planetary Sciences',
 'kmeans_13': 'Arts and Humanities',
 'kmeans_14': 'Engineering',
 'kmeans_15': 'Mathematics',
 'kmeans_16': 'Veterinary',
 'kmeans_17': 'Biochemistry, Genetics and Molecular Biology',
 'kmeans_18': 'Medicine',
 'kmeans_19': 'Social Sciences',
 'kmeans_20': 'Medicine',
 'kmeans_21': 'Agricultural and Biological Sciences',
 'kmeans_22': 'Medicine',
 'kmeans_23': 'Social Sciences',
 'kmeans_24': 'Business, Management and Accounting',
 'kmeans_25': 'Social Sciences'}

# final color code

In [99]:
node_color = node_appearance_order.copy()
for i, label in enumerate(node_appearance_order):
    if label in method12Scopus:
        node_color[i] = Scopus2color[method12Scopus[label]]
    elif label in method22Scopus:
        node_color[i] = Scopus2color[method22Scopus[label]]
node_color

['#000099',
 '#000099',
 '#CC00FF',
 '#000099',
 '#375623',
 '#66FF66',
 '#375623',
 '#66FF66',
 '#66FF66',
 '#336699',
 '#336699',
 '#187402',
 '#187402',
 '#6C0000',
 '#9A0000',
 '#D20000',
 '#9A0000',
 '#FF5C29',
 '#FF5C29',
 '#D20000',
 '#16A90F',
 '#FBFF57',
 '#A679FF',
 '#7030A0',
 '#FE0000',
 '#FE0000',
 '#8FA329',
 '#66FF66',
 '#FF7C80',
 '#D20000',
 '#FFCC00',
 '#D26B04',
 '#CCB3FF',
 '#92D050',
 '#7030A0',
 '#7030A0',
 '#0000F2',
 '#7030A0',
 '#FC9320',
 '#FBFF57',
 '#7030A0',
 '#7030A0',
 '#7030A0',
 '#000000',
 '#0099FF',
 '#9900CC',
 '#85D6FF',
 '#FFCC00',
 '#92D050',
 '#66FF66',
 '#66FF66',
 '#CC00FF']

# Top-words

In [100]:
import pandas as pd
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

# 自定义词映射
word_mapping = {'mechanics': 'mechanic',
                'mechanical': 'mechanic',
                'electrical': 'electric',
                'electronics': 'electric',
                'financial': 'finance',
                'political': 'politics',
                'historical': 'history',
                'computer': 'computing',
                'intelligent': 'intelligence',
                'agricultural': 'agriculture',
                'educational': 'education',
                'dental': 'dentistry',
                'archaeological': 'archaeology',
                'mathematical': 'mathematics',
                'mathematica': 'mathematics',
                'matematico': 'mathematics',
                'mathematicae': 'mathematics',
                'economic': 'economics',
                'chemical': 'chemistry',
                'geophysical': 'geophysics',
                'botanical': 'botany',
                'physical': 'physics',
                'entomological': 'entomology', 
                'entomologist': 'entomology',
                'biological': 'biology',
                'geographical': 'geography',
                'geological': 'geology',
                'geographer': 'geography',
                'environmental': 'environment',
                'cells': 'cell',
                'policy': 'politics',
                'psychological': 'psychology',
                'linguistical': 'linguistics',
                'linguistica': 'linguistics',
                'medical': 'medicine',
}

# 读取数据
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s", "clustering_results", "cluster_df.parquet"))
cluster_df = cluster_df.merge(pd.read_parquet(os.path.join(get_data_dir(), "MAG_venue.parquet")), left_index=True, right_index=True)

# 定义预处理函数，进行自定义词映射
def preprocess_text(text):
    words = text.lower().split()  # 将文本小写并按空格分词
    mapped_words = [word_mapping[word] if word in word_mapping else word for word in words]
    filtered_words = [word for word in mapped_words]
    return ' '.join(filtered_words)

INFO: Pandarallel will run on 48 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
data_dir:  /home/zqlyu2/projects/periodical-clustering/data
data_dir:  /home/zqlyu2/projects/periodical-clustering/data


## Top-words for cluster label 1

In [24]:
# 按 cluster_label 分组，将 OriginalVenue 文本合并为一个文档，并进行预处理
cluster_docs1 = (
    cluster_df.groupby(f'{method1}_label').parallel_apply(lambda x: preprocess_text(' '.join(x.OriginalVenue))).tolist()
)

In [25]:
# 自定义停用词
periodical_title_stopwords = [
    'journal', 'journals', 'proceedings', 'proceeding', 'transactions', 'transaction',
    'international', 'national', 'association', 'society', 'research',
    'review', 'science', 'sciences', 'university', 'de', 'studies'
]

# 计算 TF-IDF
top_words_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english')+periodical_title_stopwords)
tfidf_matrix1 = top_words_vectorizer.fit_transform(cluster_docs1)

In [26]:
top_n = 5  # 每个聚类提取前 n 个关键词
# 获取每个聚类中最重要的关键词
top_words1 = {}
feature_names = top_words_vectorizer.get_feature_names_out()

# 确保 cluster_label 的顺序与 cluster_docs 一致
cluster_labels = sorted(cluster_df[f'{method1}_label'].unique())

for j, cluster_label in enumerate(cluster_labels):
    tfidf_scores = tfidf_matrix1[j].toarray().flatten()
    top_indices = tfidf_scores.argsort()[-top_n:][::-1]  # 获取前 n 个关键词的索引
    top_words1[f"{method1}_"+str(cluster_label)] = [feature_names[idx] for idx in top_indices]

top_words1

{'Scopus_Agricultural and Biological Sciences': ['plant',
  'food',
  'entomology',
  'agriculture',
  'ecology'],
 'Scopus_Arts and Humanities': ['history',
  'studies',
  'philosophy',
  'religion',
  'literature'],
 'Scopus_Biochemistry, Genetics and Molecular Biology': ['cell',
  'biology',
  'molecular',
  'genetics',
  'biotechnology'],
 'Scopus_Business, Management and Accounting': ['management',
  'business',
  'marketing',
  'accounting',
  'tourism'],
 'Scopus_Chemical Engineering': ['catalysis',
  'chemistry',
  'engineering',
  'technology',
  'thermal'],
 'Scopus_Chemistry': ['chemistry',
  'organic',
  'spectrometry',
  'spectroscopy',
  'analytical'],
 'Scopus_Computer Science': ['computing',
  'information',
  'systems',
  'intelligence',
  'ieee'],
 'Scopus_Decision Sciences': ['operations',
  'management',
  'logistics',
  'operational',
  'chain'],
 'Scopus_Dentistry': ['dentistry',
  'oral',
  'orthodontics',
  'maxillofacial',
  'surgery'],
 'Scopus_Earth and Plane

## Top-words for cluster label 2

In [101]:
# 按 cluster_label 分组，将 OriginalVenue 文本合并为一个文档，并进行预处理
cluster_docs2 = (
    cluster_df.groupby(f'{method2}_label').parallel_apply(lambda x: preprocess_text(' '.join(x.OriginalVenue))).tolist()
)

In [102]:
# 计算 TF-IDF
top_words_vectorizer = TfidfVectorizer(stop_words=stopwords.words('english')+periodical_title_stopwords)
tfidf_matrix2 = top_words_vectorizer.fit_transform(cluster_docs2)

In [103]:
top_n = 5  # 每个聚类提取前 n 个关键词
# 获取每个聚类中最重要的关键词
top_words2 = {}
feature_names = top_words_vectorizer.get_feature_names_out()

# 确保 cluster_label 的顺序与 cluster_docs 一致
cluster_labels = sorted(cluster_df[f'{method2}_label'].unique())

for j, cluster_label in enumerate(cluster_labels):
    tfidf_scores = tfidf_matrix2[j].toarray().flatten()
    top_indices = tfidf_scores.argsort()[-top_n:][::-1]  # 获取前 n 个关键词的索引
    top_words2[f"{method2}_"+str(cluster_label)] = [feature_names[idx] for idx in top_indices]

top_words2

{'kmeans_0': ['law', 'politics', 'studies', 'economics', 'european'],
 'kmeans_1': ['geography',
  'environmental',
  'economics',
  'politics',
  'planning'],
 'kmeans_2': ['computing', 'systems', 'information', 'acm', 'ieee'],
 'kmeans_3': ['dentistry', 'oral', 'surgery', 'ophthalmology', 'dermatology'],
 'kmeans_4': ['oncology', 'cancer', 'surgery', 'clinical', 'gastroenterology'],
 'kmeans_5': ['ieee', 'electric', 'systems', 'engineering', 'computing'],
 'kmeans_6': ['physics', 'ieee', 'electric', 'materials', 'optics'],
 'kmeans_7': ['medicine', 'clinical', 'cardiovascular', 'cardiology', 'heart'],
 'kmeans_8': ['ecology', 'entomology', 'biology', 'zoology', 'conservation'],
 'kmeans_9': ['psychology', 'psychiatry', 'mental', 'social', 'child'],
 'kmeans_10': ['chinese', 'technology', 'engineering', 'information', 'laser'],
 'kmeans_11': ['chemistry',
  'materials',
  'polymer',
  'engineering',
  'catalysis'],
 'kmeans_12': ['geology', 'earth', 'geophysics', 'water', 'engineering

# Final flow data

In [104]:
# 读取数据
cluster_df = pd.read_parquet(os.path.join(get_data_dir(), "2010s", 'clustering_results', "cluster_df.parquet"))
method1, method2 = "Scopus", "kmeans"

# 计算每个method1_label类别的样本数的10%
method1_counts = cluster_df.groupby(f"{method1}_label").size()
min_thresholds = {label: int(count * 0.1) for label, count in method1_counts.items()}

# 统计流量信息，按 Scopus_label 和 skm_label 分组计数
flow_data = (
    cluster_df.groupby([f"{method1}_label", f"{method2}_label"])
    .size()
    .reset_index(name="flow")
    .apply(lambda row: [row[f"{method1}_label"], str(row[f"{method2}_label"]), row["flow"]], axis=1)
    .tolist()
)

# 过滤流量数据，只保留大于对应method1_label阈值的条目
filtered_flow_data = [entry for entry in flow_data if entry[2] > min(min_thresholds[entry[0]], 50)]
for entry in filtered_flow_data:
    if method1 == "Scopus":
        entry[0] = str(entry[0])
        entry[1] = "#"+str(entry[1])+': '+', '.join(top_words2[f"{method2}_"+entry[1]])
    else:
        entry[0] = f"{method1}_{str(entry[0])}: "+','.join(top_words1[f"{method1}_"+str(entry[0])])
        entry[1] = f"{method2}_{str(entry[1])}: "+','.join(top_words2[f"{method2}_"+entry[1]])
filtered_flow_data

data_dir:  /home/zqlyu2/projects/periodical-clustering/data


[['Agricultural and Biological Sciences',
  '#8: ecology, entomology, biology, zoology, conservation',
  616],
 ['Agricultural and Biological Sciences',
  '#16: veterinary, animal, parasitology, veterinaria, aquaculture',
  104],
 ['Agricultural and Biological Sciences',
  '#21: plant, food, agriculture, biotechnology, biology',
  526],
 ['Arts and Humanities',
  '#0: law, politics, studies, economics, european',
  54],
 ['Arts and Humanities',
  '#13: studies, history, archaeology, literature, philosophy',
  1107],
 ['Arts and Humanities',
  '#19: linguistics, language, english, studies, teaching',
  56],
 ['Arts and Humanities',
  '#23: studies, social, culture, sociology, communication',
  202],
 ['Biochemistry, Genetics and Molecular Biology',
  '#17: cell, biology, molecular, medicine, immunology',
  538],
 ['Business, Management and Accounting',
  '#24: economics, management, business, finance, accounting',
  591],
 ['Chemical Engineering',
  '#11: chemistry, materials, polymer, 

# Checking periodicals in every flow

In [ ]:
cluster_df = cluster_df.merge(pd.read_parquet(os.path.join(get_data_dir(), "MAG_venue.parquet"))['OriginalVenue'], left_index=True, right_index=True)

data_dir:  /home/lyuzhuoqi/projects/clustering/data


In [ ]:
cluster_df[(cluster_df['movMF_label'] == 24) & (cluster_df['skm_label'] == 3)].sample(20)['OriginalVenue'].tolist()

['Journal of Social Ontology',
 'Acta Theologica',
 'Scientia et Fides',
 'Acta Musicologica',
 'Estudos Históricos',
 'Berichte Zur Wissenschaftsgeschichte',
 'Savoirs Et Clinique',
 'Anuario De Estudios Medievales',
 "International Research in Children's Literature",
 'Adaptation',
 'Journal of Visual Culture',
 'Technoetic arts',
 'Synthesis Philosophica',
 'Muziki',
 'American Imago',
 'Philosophy of Science',
 'Revista de Medicina y Cine / Journal of Medicine and Movies',
 'Notes and Records',
 'Arbor-ciencia Pensamiento Y Cultura',
 'Implicit Religion']